In [7]:
# Acessando informações do arquivo
arquivo = open("entrada.txt", "r")
linha = arquivo.readline()
linhas = []
while linha != "":
    linhas.append(linha)
    linha = arquivo.readline()
arquivo.close()


print(linhas)

['3\n', '2 a b\n', '1\n', '1 2\n', '6\n', '0 a 1\n', '0 b 1\n', '1 a 1\n', '1 b 2\n', '2 a 0\n', '2 b 2\n', '10\n', 'abbbba\n', 'aabbbb\n', 'bbabbabbabbb\n', 'bbbbbbbbbbb\n', '-\n', 'abababababab\n', 'bbbbaabbbb\n', 'abba\n', 'a\n', 'aaa']


In [10]:
# Atribuindo informações para autômato finito
def get_info(linha):
  return linha.replace('\n', '')

class State:
  def __init__(self, state):
    self.state = state
    self.is_initial = False
    self.is_acceptance = False
    self.transitions = []

  def set_as_initial(self):
    self.is_initial = True

  def set_as_acceptance(self):
    self.is_acceptance = True;

  def add_transition(self, transition):
    self.transitions.append(transition)

  def get_transition(self, terminal):
    next_states = []
    lambda_states = []
    for transition in self.transitions:
      if transition.terminal == terminal and terminal != '-':
        next_states.append(transition.next_state)
      elif transition.terminal == '-' :
        lambda_states.append(transition.next_state)

    return next_states, lambda_states

class Transition:
  def __init__(self, transition):
    transition_info = transition.split(' ')
    self.current_state = int(transition_info[0])
    self.terminal = transition_info[1]
    self.next_state = int(transition_info[2])

class Finite_Automaton:
  def __init__(self, linhas, verbose=True):
    try:
        #Número de estados
        state_number = int(get_info(linhas[0]))
        self.states = []
        for i in range(0, state_number):
          state = State(i)
          self.states.append(state)

        #Terminais
        terminals_info = get_info(linhas[1]).split(' ')
        terminal_quantity = int(terminals_info[0])
        self.terminals = []
        for i in range(1, terminal_quantity+1):
          self.terminals.append(terminals_info[i])

        #Estados iniciais
        self.initial_states_amount = int(get_info(linhas[2]))
        self.initial_states = []
        for i in range(0, self.initial_states_amount):
          self.states[i].set_as_initial()
          self.initial_states.append(i)

        #Estados de aceitação
        acceptance_info = get_info(linhas[3]).split(' ')
        acceptance_quantity = int(acceptance_info[0])
        self.acceptance_states = []
        for i in range(1, acceptance_quantity+1):
          self.states[int(acceptance_info[i])].set_as_acceptance()
          self.acceptance_states.append(int(acceptance_info[i]))

        #Transições
        transitions_amount = int(get_info(linhas[4]))
        current_line = 4
        for i in range(0, transitions_amount):
          current_line += 1
          transition = Transition(get_info(linhas[current_line]))
          state = int(get_info(linhas[current_line]).split(' ')[0])
          self.states[state].add_transition(transition)

        #Cadeias para teste
        current_line += 1
        test_cases_amount = int(get_info(linhas[current_line]))
        self.test_cases = []
        for i in range(0, test_cases_amount):
          current_line += 1
          self.test_cases.append(get_info(linhas[current_line]))

    except:
        print("Arquivo de entrada incorreto")

    self.verbose = verbose
    # Resumo do autômato
    if verbose:
      self.get_summary()

  def generate_output(self):
    #Escrever arquivo de saída
    output = open("saida.txt", "w")

    for chain in self.test_cases:
      result = False
      try:
        result = self.test_chain(chain)
      except:
        "Algum erro occoreu, verifique se as entradas estão corretas"
      if result:
        output.write("aceita\n")
      else:
        output.write("rejeita\n")
    output.close()


  def test_chain(self, chain):
    #Testa cadeia atual
    chain_length = len(chain)

    if self.verbose:
      print("==================")
      print("Iniciando teste para cadeia:", chain)
      print("Estados iniciais:", self.initial_states)
    for initial_state in self.initial_states:
      resultado = self.test_current_terminal(chain, 0, 1 == len(chain), self.states[initial_state])
      if resultado:
        break;
    if self.verbose:
      print("Resultado:", resultado)
    return resultado

  def test_current_terminal(self, chain, position, is_last_terminal, current_state):
    current_terminal, is_lambda = self.get_character(chain, position)
    possible_next_states, next_lambda_states = current_state.get_transition(current_terminal)
    result = False
    if self.verbose:
      print(" "*position, "delta(" + str(current_state.state) + "," + str(current_terminal) + ") =",possible_next_states + next_lambda_states)
      print(" "*position, "is_lambda:", is_lambda)
      print(" "*position, "is_last_terminal:", is_last_terminal)

    if is_last_terminal:
      if not is_lambda:
        for next_state in possible_next_states:
          if self.states[next_state].is_acceptance:
            return True
      else:
        if current_state.is_acceptance:
          return True

    position += 1
    for next_state in possible_next_states:
      resultado = self.test_current_terminal(chain, position, position >= len(chain)-1, self.states[next_state])
      if resultado:
        return True

    for next_state in next_lambda_states:
      resultado = self.test_current_terminal(chain, position, position >= len(chain)-1, self.states[next_state])
      if resultado:
        return True

    return result

  def get_character(self, chain, position):
    if position >= len(chain):
      return '-', True

    current_terminal = chain[position]
    if current_terminal in self.terminals or current_terminal == '-':
      is_lambda = current_terminal == '-'
      return current_terminal, is_lambda

    return None

  def get_summary(self):
    print("Terminais: " + ', '.join(self.terminals))
    print("Estados iniciais: " + '[' + ', '.join(str(initial_state) for initial_state in self.initial_states) + ']')
    print("Estados de aceitação: " + '[' +  ', '.join(str(acceptance_state) for acceptance_state in self.acceptance_states) + ']')
    print("Número de estados: " + str(len(self.states)))
    for i in range(0, len(self.states)):
      print(" Transições do Estado " + str(self.states[i].state) +":")
      for j in range(0, len(self.states[i].transitions)):
        print("   delta(" + str(self.states[i].transitions[j].current_state) + ", " + self.states[i].transitions[j].terminal + ") = " + str(self.states[i].transitions[j].next_state))

automaton = Finite_Automaton(linhas, verbose=True)
automaton.generate_output()

Terminais: a, b
Estados iniciais: [0]
Estados de aceitação: [2]
Número de estados: 3
 Transições do Estado 0:
   delta(0, a) = 1
   delta(0, b) = 1
 Transições do Estado 1:
   delta(1, a) = 1
   delta(1, b) = 2
 Transições do Estado 2:
   delta(2, a) = 0
   delta(2, b) = 2
Iniciando teste para cadeia: abbbba
Estados iniciais: [0]
 delta(0,a) = [1]
 is_lambda: False
 is_last_terminal: False
  delta(1,b) = [2]
  is_lambda: False
  is_last_terminal: False
   delta(2,b) = [2]
   is_lambda: False
   is_last_terminal: False
    delta(2,b) = [2]
    is_lambda: False
    is_last_terminal: False
     delta(2,b) = [2]
     is_lambda: False
     is_last_terminal: False
      delta(2,a) = [0]
      is_lambda: False
      is_last_terminal: True
       delta(0,-) = []
       is_lambda: True
       is_last_terminal: True
Resultado: False
Iniciando teste para cadeia: aabbbb
Estados iniciais: [0]
 delta(0,a) = [1]
 is_lambda: False
 is_last_terminal: False
  delta(1,a) = [1]
  is_lambda: False
  is_las